In [1]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
from pypardiso import spsolve
from pathlib import Path
from gsa_framework.utils import read_pickle, write_pickle
import sys
sys.path.append('/Users/akim/PycharmProjects/akula')

# Local files
from akula.sensitivity_analysis.local_sa import *

project = 'GSA for archetypes'
bd.projects.set_current(project)
const_factor = 10
cutoff = 1e-3  # For contribution analysis
max_calc = 1e3  # For supply chain traversal

# Setups

In [2]:
co = bd.Database('swiss consumption 1.0')
fu = [act for act in co if "average consumption" in act['name']][0]

write_dir = Path("write_files") / project.lower().replace(" ", "_") / fu['name'].lower().replace(" ", "_")
write_dir_sct = write_dir / "supply_chain_traversal" 
write_dir_sct.mkdir(exist_ok=True, parents=True)

demand = {fu: 1}
method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
fu_mapped, packages, _ = bd.prepare_lca_inputs(demand=demand, method=method, remapping=False)
lca = bc.LCA(demand=fu_mapped, data_objs=packages)
lca.lci()
lca.lcia()
lca.score

1094.9290193609536

In [3]:
# Get all relevant data
ei = bd.Database('ecoinvent 3.8 cutoff').datapackage()
co = bd.Database('swiss consumption 1.0').datapackage()
cf = bd.Method(method).datapackage()

# Technosphere
ei_tech = ei.filter_by_attribute('matrix', 'technosphere_matrix')
co_tech = co.filter_by_attribute('matrix', 'technosphere_matrix')
get_tech_resource_kind = lambda kind: np.hstack(
    [
        ei_tech.get_resource(f'ecoinvent_3.8_cutoff_technosphere_matrix.{kind}')[0], 
        co_tech.get_resource(f'swiss_consumption_1.0_technosphere_matrix.{kind}')[0]
    ]
)
indices_tech = get_tech_resource_kind('indices')
data_tech = get_tech_resource_kind('data')
flip_tech = get_tech_resource_kind('flip')

# Biosphere
ei_bio = ei.filter_by_attribute('matrix', 'biosphere_matrix')
indices_bio = ei_bio.get_resource('ecoinvent_3.8_cutoff_biosphere_matrix.indices')[0]
data_bio = ei_bio.get_resource('ecoinvent_3.8_cutoff_biosphere_matrix.data')[0]
distributions_bio = ei_bio.get_resource('ecoinvent_3.8_cutoff_biosphere_matrix.distributions')[0]

# Characterization
indices_cf = cf.get_resource('IPCC_2013_climate_change_GWP_100a_uncertain_matrix_data.indices')[0]
data_cf = cf.get_resource('IPCC_2013_climate_change_GWP_100a_uncertain_matrix_data.data')[0]
distributions_cf = cf.get_resource('IPCC_2013_climate_change_GWP_100a_uncertain_matrix_data.distributions')[0]

In [4]:
# Get technosphere uncertainty boolean array
distributions_ei = ei_tech.get_resource(f'ecoinvent_3.8_cutoff_technosphere_matrix.distributions')[0] 
has_uncertainty_ei = distributions_ei['uncertainty_type'] >= 2

indices_tech_co = co_tech.get_resource(f'swiss_consumption_1.0_technosphere_matrix.indices')[0]
has_uncertainty_dict = {}
for act in bd.Database('swiss consumption 1.0'):
    exchanges = list(act.exchanges())
    col = lca.dicts.activity[act.id]
    for exc in exchanges:
        if exc.get('has_uncertainty', False):
            row = lca.dicts.activity[exc.input.id]
            has_uncertainty_dict[(exc.input.id, act.id)] = True
has_uncertainty_co = np.array([has_uncertainty_dict.get(tuple(ids), False) for ids in indices_tech_co])

has_uncertainty_tech = np.hstack(
    [
        has_uncertainty_ei,
        has_uncertainty_co,
    ]
)

# Step 1. Remove non influential with contribution analysis

## Step 1.1 Technosphere & Supply chain traversal

In [5]:
%%time
fp_sct = write_dir_sct / f"sct.cutoff_{cutoff:.0e}.maxcalc_{max_calc:.0e}.pickle"
if fp_sct.exists():
    use_indices_tech_without_noninf = read_pickle(fp_sct)
else:    
    use_indices_tech_without_noninf = get_inds_tech_without_noninf(lca, cutoff, max_calc)
    write_pickle(use_indices_tech_without_noninf, fp_sct)

fp_mask_without_noninf = write_dir / f"mask.tech.without_noninf.cutoff_{cutoff:.0e}.maxcalc_{max_calc:.0e}.pickle"
if fp_mask_without_noninf.exists():
    mask_tech_without_noninf = read_pickle(fp_mask_without_noninf)
else:
    mask_tech_without_noninf = get_mask(indices_tech, use_indices_tech_without_noninf)
    write_pickle(mask_tech_without_noninf, fp_mask_without_noninf)

CPU times: user 830 µs, sys: 1.26 ms, total: 2.09 ms
Wall time: 3.01 ms


## Step 1.2 Biosphere

In [6]:
use_indices_bio_without_noninf = get_inds_bio_without_noninf(lca, cutoff)
fp_mask_bio_without_noninf = write_dir / f"mask.bio.without_noninf.cutoff_{cutoff:.0e}.pickle"
if fp_mask_bio_without_noninf.exists():
    mask_bio_without_noninf = read_pickle(fp_mask_bio_without_noninf)
else:
    mask_bio_without_noninf = get_mask(indices_bio, use_indices_bio_without_noninf)
    write_pickle(mask_bio_without_noninf, fp_mask_bio_without_noninf)

## Step 1.3 Characterization

In [7]:
use_indices_cf_without_noninf = get_inds_cf_without_noninf(lca, cutoff)
fp_mask_cf_without_noninf = write_dir / f"mask.cf.without_noninf.cutoff_{cutoff:.0e}.pickle"
if fp_mask_cf_without_noninf.exists():
    mask_cf_without_noninf = read_pickle(fp_mask_cf_without_noninf)
else:
    mask_cf_without_noninf = get_mask(indices_cf, use_indices_cf_without_noninf)
    write_pickle(mask_cf_without_noninf, fp_mask_cf_without_noninf)

# Step 2. Run local SA

## 2.1 Technosphere

In [8]:
def run_local_sa_tech(
    matrix_type,
    fu_mapped,
    packages,
    indices_tech,
    data_tech,
    has_uncertainty_tech,
    mask_tech_without_noninf,
    flip_tech,
    factors,
    cutoff,
):
    for i, factor in enumerate(factors):
        local_sa_current = run_local_sa(
            matrix_type,
            fu_mapped,
            packages,
            indices_tech,
            data_tech,
            has_uncertainty_tech,
            mask_tech_without_noninf,
            flip_tech,
            factor,
            cutoff,
        )
        if i == 0:
            local_sa = {k:np.array([v]) for k,v in local_sa_current.items()}
        else:
            local_sa = {k: np.hstack([local_sa[k], np.array([local_sa_current[k]])]) for k in local_sa.keys()}
    return local_sa
        

In [9]:
%%time
fp_local_sa_tech = write_dir / f"local_sa.tech.cutoff_{cutoff:.0e}.maxcalc_{max_calc:.0e}.pickle"
if fp_local_sa_tech.exists():
    local_sa_tech = read_pickle(fp_local_sa_tech)
else:
    local_sa_tech = run_local_sa_tech(
        "technosphere",
        fu_mapped,
        packages,
        indices_tech,
        data_tech,
        has_uncertainty_tech,
        mask_tech_without_noninf,
        flip_tech,
        [1/const_factor, const_factor],
        cutoff,
    )
    write_pickle(local_sa_tech, fp_local_sa_tech)

CPU times: user 1.04 ms, sys: 1.22 ms, total: 2.26 ms
Wall time: 2.24 ms


In [10]:
local_sa_tech

{(4435, 7979): array([1088.7807232 , 1156.41198222]),
 (4563, 5504): array([1090.5244405 , 1139.10229337]),
 (4935, 5504): array([1057.49102283, 1473.99269434]),
 (5504, 15411): array([1048.75923401, 1560.14582465]),
 (5824, 20020): array([1090.80613998, 1136.82580721]),
 (5887, 7979): array([1089.85588614, 1145.66035272]),
 (6224, 4891): array([1093.62786224, 1107.99334499]),
 (7051, 5504): array([1093.65194585, 1107.70578572]),
 (7302, 17206): array([1056.49745377, 1479.24469631]),
 (7979, 7134): array([1060.86666134, 1435.55261114]),
 (8107, 20020): array([1093.59780062, 1108.30936092]),
 (8307, 5504): array([1092.10683612, 1123.18859614]),
 (8595, 20020): array([1093.61754981, 1108.10982611]),
 (8613, 13455): array([1060.11441688, 1443.07504483]),
 (8635, 12329): array([1087.81974631, 1167.47060612]),
 (8974, 20020): array([1093.57835498, 1108.50553573]),
 (9164, 13648): array([1093.74115407, 1106.80767212]),
 (9649, 20020): array([1093.7832018 , 1106.43861196]),
 (9884, 6128): arr

In [ ]:
# if fp_mask_tech_without_lowinf.exists():
#     mask_tech_without_lowinf = read_pickle(fp_mask_tech_without_lowinf)
# else:
#     mask_tech_without_lowinf = get_mask(indices_tech, use_indices_tech_without_lowinf)
#     write_pickle(mask_tech_without_lowinf, fp_mask_tech_without_lowinf)

## 2.2 Biosphere

In [ ]:
%%time
fp_local_sa_bio = write_dir / f"local_sa.bio.cutoff_{cutoff:.0e}.pickle"
if fp_local_sa_bio.exists():
    local_sa_bio = read_pickle(fp_mask_bio_without_lowinf)
else:
    local_sa_bio = run_local_sa(
        "biosphere",
        fu_mapped,
        packages,
        indices_bio,
        data_bio,
        distributions_bio,
        mask_bio_without_noninf,
        None,
        const_factor,
        cutoff,
    )
    write_pickle(local_sa_bio, fp_local_sa_bio)
    
#     mask_bio_without_lowinf = get_mask(indices_bio, use_indices_bio_without_lowinf)
#     assert sum(mask_bio_without_lowinf) == len(use_indices_bio_without_lowinf)
#     mask_bio_without_lowinf = get_mask(indices_bio, use_indices_bio_without_lowinf)
#     write_pickle(mask_bio_without_lowinf, fp_mask_bio_without_lowinf)

## 2.3 Characterization

In [ ]:
%%time
fp_local_sa_cf = write_dir / f"local_sa.cf.cutoff_{cutoff:.0e}.pickle"
if fp_local_sa_cf.exists():
    local_sa_cf = read_pickle(fp_local_sa_cf)
else:
    local_sa_cf = run_local_sa(
        "characterization",
        fu_mapped,
        packages,
        indices_cf,
        data_cf,
        distributions_cf,
        mask_cf_without_noninf,
        None,
        const_factor,
        cutoff,
    )
    write_pickle(local_sa_cf, fp_local_sa_cf)
#     mask_cf_without_lowinf = get_mask(indices_cf, use_indices_cf_without_lowinf)
#     assert sum(mask_cf_without_lowinf) == len(use_indices_cf_without_lowinf)
#     mask_cf_without_lowinf = get_mask(indices_cf, use_indices_cf_without_lowinf)
#     write_pickle(mask_cf_without_lowinf, fp_mask_cf_without_lowinf)